In [13]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans

In [18]:
df = pd.read_csv("penguins.csv")
df = df.dropna()

In [19]:
df.head()

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,39.1,18.7,181.0,3750.0,MALE
1,39.5,17.4,186.0,3800.0,FEMALE
2,40.3,18.0,195.0,3250.0,FEMALE
4,36.7,19.3,193.0,3450.0,FEMALE
5,39.3,20.6,190.0,3650.0,MALE


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335 entries, 0 to 343
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   culmen_length_mm   335 non-null    float64
 1   culmen_depth_mm    335 non-null    float64
 2   flipper_length_mm  335 non-null    float64
 3   body_mass_g        335 non-null    float64
 4   sex                335 non-null    object 
dtypes: float64(4), object(1)
memory usage: 15.7+ KB


In [21]:
features = df[["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm", "body_mass_g"]]

In [22]:
output = df["sex"]

In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    features, output, test_size=0.30, random_state=42)

In [24]:
model = KMeans(n_clusters=3)

In [26]:
model.fit(X_train) 

KMeans(n_clusters=3)

In [27]:
model.cluster_centers_

array([[  45.24473684,   16.70131579,  201.03947368, 4502.96052632],
       [  41.52162162,   18.08648649,  190.36036036, 3535.58558559],
       [  48.75531915,   15.49787234,  219.89361702, 5418.61702128]])